In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os
import pandas_profiling

#to get all columns in dataframe on page
pd.set_option('display.max_columns', None)

import PyPDF2

from scipy.stats import kurtosis
from scipy.stats import skew

from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler

In [ ]:
#listing and evaluating each file
for f in os.listdir():
    print(f.ljust(30) +"--" + str(round(os.path.getsize(f) / 1000000, 2)) + 'MB')

In [ ]:
filename = 'Data - parkinsons.names'
file = open(filename,mode='r')
text = file.read()
file.close()
print(text)

- This document contains the details on data set

In [ ]:

filename = 'LETTER G - Z.pdf'
# creating an object 
file = open(filename, 'rb')

# creating a pdf reader object
fileReader = PyPDF2.PdfFileReader(file)

# print the number of pages in pdf file
print(fileReader.numPages)

In [ ]:
pageObj = fileReader.getPage(0)
pageObj.extractText()

In [ ]:
for i in range(fileReader.numPages):
    pageObj = fileReader.getPage(i)
    print(pageObj.extractText())

In [ ]:
from IPython.display import IFrame
IFrame(filename, width=600, height=900)

- This pdf contains the letter details

In [ ]:
data_p=pd.read_csv('Data - Parkinsons')
print("Shape:",data_p.shape)
print("Contains 195 rows and 24 columns")
data_p.head(10)

In [ ]:
data_p.tail(10)

In [ ]:
df=data_p

In [ ]:
df.info()

- Total 24 columns are present
- Majority of columns are numeric and Float
- status column is int and contins 0 & 1 represnting healthy and parkinson's

#### Attribute Information:

- Matrix column entries (attributes):
- name - ASCII subject name and recording number
- MDVP:Fo(Hz) - Average vocal fundamental frequency
- MDVP:Fhi(Hz) - Maximum vocal fundamental frequency
- MDVP:Flo(Hz) - Minimum vocal fundamental frequency
- MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP - Several measures of variation in fundamental frequency
- MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA - Several measures of variation in amplitude
- NHR,HNR - Two measures of ratio of noise to tonal components in the voice
- RPDE,D2 - Two nonlinear dynamical complexity measures
- DFA - Signal fractal scaling exponent
- spread1,spread2,PPE - Three nonlinear measures of fundamental frequency variation 

#### Target Column
- status - Health status of the subject (one) - Parkinson's, (zero) - healthy

In [ ]:
#df.describe(include='all')
df.describe().T

- The rows of the CSV file contain an instance corresponding to one voice recording. 
- There are around six recordings per patient, the name of the patient is identified in the first column
- Column values in tables are varying in scale and ranging from negative to positive
- All column values shows real numbers and no abnormal details appeared so far
- Name column is of not much importance for prediction

In [ ]:
df.status.value_counts()

In [ ]:
df.status.value_counts(1)*100

- 75% of data is related for patient having Parkinson's

In [ ]:
#Rearranging - moving status column towards end of file
stat=df.pop('status') 
df['status'] = stat

### Univariate Analysis

- Analyzing Frequencies
- MDVP:Fo(Hz) - Average vocal fundamental frequency
- MDVP:Fhi(Hz) - Maximum vocal fundamental frequency
- MDVP:Flo(Hz) - Minimum vocal fundamental frequency

In [ ]:
f, axes = plt.subplots(1, 3, figsize=(15,8), sharey='row')

sns.distplot(df['MDVP:Flo(Hz)'],ax=axes[0],axlabel="MDVP:Flo(Hz)-Min-" + "skew:" + str(skew(df['MDVP:Flo(Hz)'])))
sns.distplot(df['MDVP:Fo(Hz)'],ax=axes[1],axlabel="MDVP:Fo(Hz)-Avg-" + "skew:" +str(skew(df['MDVP:Fo(Hz)'])))
sns.distplot(df['MDVP:Fhi(Hz)'],ax=axes[2],axlabel="MDVP:Fhi(Hz)-Max-" + "skew:" +str(skew(df['MDVP:Fhi(Hz)'])))
plt.show()

- Maximum vocal fundamental frequency is positively skewed, majority of the values are around 100-200 but few outliers beyond 300 is dragging its tail and making it poitively skewed. High outliers presence
- Average vocal fundamental frequency is almost symmetrical with skewness around 0.6. Highest peak concentrtion of data is around 100-150
- Minimum vocal fundamental frequency is comparativley more positivley skewed then average and less skewed than max. Majority of values are between 75-150

- Analyzing
- MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP - Several measures of variation in fundamental frequency

In [ ]:
f, axes = plt.subplots(2,3,figsize=(15,8)) 
sns.distplot(df['MDVP:Jitter(%)'],ax=axes[0,0],axlabel="MDVP:Jitter(%)-" + "skew:" + str(skew(df['MDVP:Jitter(%)'])) )
sns.distplot(df['MDVP:Jitter(Abs)'],ax=axes[0,1],axlabel="MDVP:Shimmer(dB)-" + "skew:" + str(skew(df['MDVP:Jitter(Abs)'])) )
sns.distplot(df['MDVP:RAP'],ax=axes[0,2],axlabel="Shimmer:APQ3-" + "skew:" + str(skew(df['MDVP:RAP'])) )
sns.distplot(df['MDVP:PPQ'],ax=axes[1,0],axlabel="Shimmer:APQ5-" + "skew:" + str(skew(df['MDVP:PPQ'])) )
sns.distplot(df['Jitter:DDP'],ax=axes[1,1],axlabel="MDVP:APQ-" + "skew:" + str(skew(df['Jitter:DDP']))) 


- Positive skewness present in all columns in grpahs present
- No major peaks present in longer tails representing the values are very less
- HIgh outliers presence in all

- Analyzing 
- MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA - Several measures of variation in amplitude


In [ ]:
f, axes = plt.subplots(2,3,figsize=(15,8)) 
sns.distplot(df['MDVP:Shimmer'],ax=axes[0,0],axlabel="MDVP:Shimmer-" + "skew:" + str(skew(df['MDVP:Shimmer'])) )
sns.distplot(df['MDVP:Shimmer(dB)'],ax=axes[0,1],axlabel="MDVP:Shimmer(dB)-" + "skew:" + str(skew(df['MDVP:Shimmer(dB)'])) )
sns.distplot(df['Shimmer:APQ3'],ax=axes[0,2],axlabel="Shimmer:APQ3-" + "skew:" + str(skew(df['Shimmer:APQ3'])) )
sns.distplot(df['Shimmer:APQ5'],ax=axes[1,0],axlabel="Shimmer:APQ5-" + "skew:" + str(skew(df['Shimmer:APQ5'])) )
sns.distplot(df['MDVP:APQ'],ax=axes[1,1],axlabel="MDVP:APQ-" + "skew:" + str(skew(df['MDVP:APQ']))) 
sns.distplot(df['Shimmer:DDA'],ax=axes[1,2],axlabel="Shimmer:DDA-" + "skew:" + str(skew(df['Shimmer:DDA'])))

- Positive skewness present in all columns in grpahs present
- No major peaks present in longer tails representing the values are very less
- majority of peak number of values are close to zero
- High outliers presence in all

- Analyzing NHR & HNR - Noises
- NHR,HNR - Two measures of ratio of noise to tonal components in the voice

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(15,8), sharey='row')

sns.distplot(df['NHR'],ax=axes[0],axlabel="NHR-" + "skew:" + str(skew(df['NHR'])))
sns.distplot(df['HNR'],ax=axes[1],axlabel="HNR-" + "skew:" +str(skew(df['HNR'])))
plt.show()

In [ ]:
#same y-scale is not providing proper results
f, axes = plt.subplots(1, 2, figsize=(15,8))

sns.distplot(df['NHR'],ax=axes[0],axlabel="NHR-" + "skew:" + str(skew(df['NHR'])))
sns.distplot(df['HNR'],ax=axes[1],axlabel="HNR-" + "skew:" +str(skew(df['HNR'])))
plt.show()

- for NHR, high positive skewness
- but the peaks are very low representing not a lot of values are present on those range of values
- majority of values are rangig between 0.00-0.05
- More outlier presence for NHR compared to HNR
- HNR has slight negetive skewness but overll looks normally ditributed


- Analyzing RPDE,D2 
- Two nonlinear dynamical complexity measures

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(15,8))

sns.distplot(df['RPDE'],ax=axes[0],axlabel="RPDE-" + "skew:" + str(skew(df['RPDE'])))
sns.distplot(df['D2'],ax=axes[1],axlabel="D2-" + "skew:" +str(skew(df['D2'])))
plt.show()

- Both columns are less skewed and have close to normal distribution

- Analyzing DFA - Signal fractal scaling exponent

In [ ]:
sns.distplot(df['DFA'],axlabel="DFA-" + "skew:" + str(skew(df['DFA'])))

- Distribution is very less skewed and have close to normal distribution

- Analyzing spread1,spread2,PPE 
- Three nonlinear measures of fundamental frequency variation

In [ ]:
f, axes = plt.subplots(1, 3, figsize=(15,8))

sns.distplot(df['spread1'],ax=axes[0],axlabel="spread1-" + "skew:" + str(skew(df['spread1'])))
sns.distplot(df['spread2'],ax=axes[1],axlabel="spread2-" + "skew:" +str(skew(df['spread2'])))
sns.distplot(df['PPE'],ax=axes[2],axlabel="PPE-" + "skew:" +str(skew(df['PPE'])))
plt.show()

- skewness between0-1 for all. Almost normally distributed
- slight outlier presence in PPE and spread1

#### Bivariate Ananlysis

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(df.corr(), annot=True, fmt='.2f')

- from correlation details we can observe almost all the attributes are ranging between 0.2-0.5 in positive side and 2 negative correlation at -0.38
- MDVP:Fo & Flo are having same impact on Status of -0.38
- similarly all attributes related to Jitter are showing similar impact on status
- Similarly all atrributes related to Shimmer are showing similar behaviour with status
- There is a high correlation between attributes related to Jitter and Shimmer, you can observe the light & dark orange square formations for these columns in heatmap
- HNR has high -ve correlation with all attributes related to Jitter and Shimmer
- Highest positive/negative correlation noticed is with spread1,PPE and spread2

In [ ]:
g = sns.pairplot(df.drop(['name'],axis=1), hue="status", palette="husl")

- Considering all the fields values with status, we can notice two distinguished peaks but are close mostly signifying the mode values for status are close by 
- Majority of fields has significant overlap area visible for status 0 and 1 in the kde plots
- We have clear indicators in many attributes that low and high values are belonging to a certain group rather then covered by both status 0 and 1. This indicates the outliers or high or low values usually belong to one group rather than both making it ambigous.

- Analyzing Frequencies
- MDVP:Fo(Hz) - Average vocal fundamental frequency
- MDVP:Fhi(Hz) - Maximum vocal fundamental frequency
- MDVP:Flo(Hz) - Minimum vocal fundamental frequency

In [ ]:
f, axes = plt.subplots(1, 3, figsize=(15,8))

sns.boxplot(x=df['status'],y=df['MDVP:Fo(Hz)'],ax=axes[0])
sns.boxplot(x=df['status'],y=df['MDVP:Flo(Hz)'],ax=axes[1])
sns.boxplot(x=df['status'],y=df['MDVP:Fhi(Hz)'],ax=axes[2])


plt.show()

In [ ]:
#Viewing on same Y axis
f, axes = plt.subplots(1, 3, figsize=(15,8),sharey='row')

sns.boxplot(x=df['status'],y=df['MDVP:Fo(Hz)'],ax=axes[0])
sns.boxplot(x=df['status'],y=df['MDVP:Flo(Hz)'],ax=axes[1])
sns.boxplot(x=df['status'],y=df['MDVP:Fhi(Hz)'],ax=axes[2])


plt.show()

- Considering both the boxplots
- There is a significant variation in median for status 0 and 1 for  Averagea and Maximum vocal frequency.
- Median for Average on staus 0 is close to 200 and for status 1 is around 150
- Median for Maximum on staus 0 is close to 250 and for status 1 is around 150
- Few outliers presence for Maximum vocal frequencies
- For status 0 - box and whisker has much higher range compared to status 1 in all 3 vocal fundamental frequencies
- There is big overlap in range of frequencies for status 0 and 1 but majorly the higher ranges are covered with status 0
- For maximum vocal frequencies majority of outliers were belong to status 1.

- Analyzing
- MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP - Several measures of variation in fundamental frequency

In [ ]:
f, axes = plt.subplots(2,3,figsize=(15,15)) 
sns.boxplot(x=df['status'],y=df['MDVP:Jitter(%)'],ax=axes[0,0])
sns.boxplot(x=df['status'],y=df['MDVP:Jitter(Abs)'],ax=axes[0,1])
sns.boxplot(x=df['status'],y=df['MDVP:RAP'],ax=axes[0,2])
sns.boxplot(x=df['status'],y=df['MDVP:PPQ'],ax=axes[1,0])
sns.boxplot(x=df['status'],y=df['Jitter:DDP'],ax=axes[1,1]) 


- High presence of outliers in attributes realted to jitter
- For status 1 - whiskers and box were having more range and majorly all outliers are belonging to status 1.
- Majority of higher values and outliers are belonging to status 1
- Suggests the higher presence jitter is a strong indicator of Parkinson's

- Analyzing 
- MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA - Several measures of variation in amplitude

In [ ]:
f, axes = plt.subplots(2, 3, figsize=(15,15))

sns.boxplot(x=df['status'],y=df['MDVP:Shimmer'],ax=axes[0,0] )
sns.boxplot(x=df['status'],y=df['MDVP:Shimmer(dB)'],ax=axes[0,1] )
sns.boxplot(x=df['status'],y=df['Shimmer:APQ3'],ax=axes[0,2])
sns.boxplot(x=df['status'],y=df['Shimmer:APQ5'],ax=axes[1,0])
sns.boxplot(x=df['status'],y=df['MDVP:APQ'],ax=axes[1,1]) 
sns.boxplot(x=df['status'],y=df['Shimmer:DDA'],ax=axes[1,2])
plt.show()

- Noticing similar behaviour like JItter
- whiskers and boxes are having higher ranges for status 1 compared to 0
- Majority of higher values and outliers are belonging to status 1

- Analyzing NHR & HNR - Noises
- NHR,HNR - Two measures of ratio of noise to tonal components in the voice

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(15,8))

sns.boxplot(x=df['status'],y=df['NHR'],ax=axes[0])
sns.boxplot(x=df['status'],y=df['HNR'],ax=axes[1])
plt.show()

- status 1 has higher range of whisker and box comapred to status 0 for both NHR and HNR
- for NHR, majority of higher values or outliers are for status 1, beyond 0.15 all are for status 1
- for HNR as well majority of outliers towards low range values are also for status 1.

- Analyzing RPDE,D2 
- Two nonlinear dynamical complexity measures

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(15,8))

sns.boxplot(x=df['status'],y=df['RPDE'],ax=axes[0])
sns.boxplot(x=df['status'],y=df['D2'],ax=axes[1])
plt.show()

- For RPDE,  whisker and boxes are of similar range.
- For D2, majority fo outliers wre belonging to status 1

- Analyzing DFA - Signal fractal scaling exponent

In [ ]:
sns.boxplot(x=df['status'],y=df['DFA'])

- whisker is much bigger for status 1 compared to 0.
- values above .8 and below 0.6 belongs to status 1

- Analyzing spread1,spread2,PPE 
- Three nonlinear measures of fundamental frequency variation

In [ ]:
f, axes = plt.subplots(1, 3, figsize=(15,8))

sns.boxplot(x=df['status'],y=df['spread1'],ax=axes[0])
sns.boxplot(x=df['status'],y=df['spread2'],ax=axes[1])
sns.boxplot(x=df['status'],y=df['PPE'],ax=axes[2])
plt.show()

- High range values for spread1, spread2 and PPE are covered under status 1, and low range values for status 0.

### Null Values and Scaling

In [ ]:
df.isnull().sum()

In [ ]:
df.isna().sum()

- No Null/nan values found

In [ ]:
df=df.drop("name",axis=1)

In [ ]:
scaler=MinMaxScaler((-1,1))

In [ ]:
X=scaler.fit_transform( df.loc[:,df.columns != 'status'].values[:,1:])
y=df.loc[:,'status'].values
X[:,0:1]

In [ ]:
#X = df.drop("status",axis=1)
#Y = df["status"]

### Split the dataset into training and test set in the ratio of 70:30

In [ ]:
X_train, X_test, y_train,  y_test = train_test_split(X, y,train_size=0.7, random_state=10)
print(len(X_train)),print(len(X_test))

In [ ]:
y_train.shape, y_test.shape

In [ ]:
y_test[0:5]

- Data is scalled as we have lot of attributes having very diff range of values
- Data set is distributed into training and test set
- We have very less volume of data to train and even lesser volume to test

In [ ]:
from sklearn import metrics
def draw_cm( actual, predicted ):
    cm = metrics.confusion_matrix( actual, predicted, [0,1] )
    sns.heatmap(cm, annot=True,  fmt='.0f', xticklabels = ["Status 0", "Status 1"] , yticklabels = ["Status 0", "Status 1"] )
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
modelComp=pd.DataFrame()

### Train and test Models

In [ ]:
#Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
logRegModel=LogisticRegression()
logRegModel.fit(X_train,y_train)
y_predict=logRegModel.predict(X_test)

from sklearn.metrics import accuracy_score,confusion_matrix,recall_score,f1_score,precision_score,roc_curve,log_loss,auc
print('Accuracy score:',accuracy_score(y_test,y_predict))
print('confuion matrix:\n',confusion_matrix(y_test,y_predict))
print('Recall Score: ',recall_score(y_test, y_predict))
print('Precission Score: ',precision_score(y_test, y_predict))
print('F1 Score: ',f1_score(y_test, y_predict))
draw_cm(y_test, y_predict)
modelComp=pd.DataFrame({'Model':['Logistic Regression - 0.5'],'Accuracy':[accuracy_score(y_test,y_predict)*100],'Precission':[precision_score(y_test, y_predict)*100],'Recall':[recall_score(y_test, y_predict)*100]})

In [ ]:
from sklearn.preprocessing import binarize
#changing the threshold to 0.4
y_pred_class = binarize([logRegModel.predict_proba(X_test)[:, 1]], 0.4)[0]

print('Accuracy score:',accuracy_score(y_test,y_pred_class))
print('confuion matrix:\n',confusion_matrix(y_test,y_pred_class))
print('Recall Score: ',recall_score(y_test, y_pred_class))
print('Precission Score: ',precision_score(y_test, y_pred_class))
print('F1 Score: ',f1_score(y_test, y_pred_class))
draw_cm(y_test, y_pred_class)
modelComp=modelComp.append(pd.DataFrame({'Model':['Logistic Regression - 0.4'],'Accuracy':[accuracy_score(y_test,y_pred_class)*100],'Precission':[precision_score(y_test, y_pred_class)*100],'Recall':[recall_score(y_test, y_pred_class)*100]}))

In [ ]:
#KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KnnModel = KNeighborsClassifier(n_neighbors=3)
KnnModel.fit(X_train,y_train)
y_predict=KnnModel.predict(X_test)

print('Accuracy score:',accuracy_score(y_test,y_predict))
print('confuion matrix:\n',confusion_matrix(y_test,y_predict))
print('Recall Score: ',recall_score(y_test, y_predict))
print('Precission Score: ',precision_score(y_test, y_predict))
print('F1 Score: ',f1_score(y_test, y_predict))

draw_cm(y_test, y_predict)
modelComp=modelComp.append(pd.DataFrame({'Model':['KNN - 3 Neigbours'],'Accuracy':[accuracy_score(y_test,y_predict)*100],'Precission':[precision_score(y_test, y_predict)*100],'Recall':[recall_score(y_test, y_predict)*100]}))

In [ ]:
#NaiveBayes Gaussian

In [ ]:
from sklearn.naive_bayes import GaussianNB,BernoulliNB
NBGauModel = GaussianNB()

NBGauModel.fit(X_train,y_train)
y_predict=NBGauModel.predict(X_test)

print('Accuracy score:',accuracy_score(y_test,y_predict))
print('confuion matrix:\n',confusion_matrix(y_test,y_predict))
print('Recall Score: ',recall_score(y_test, y_predict))
print('Precission Score: ',precision_score(y_test, y_predict))
print('F1 Score: ',f1_score(y_test, y_predict))
draw_cm(y_test, y_predict)
modelComp=modelComp.append(pd.DataFrame({'Model':['Naive Bayes - Gaussian'],'Accuracy':[accuracy_score(y_test,y_predict)*100],'Precission':[precision_score(y_test, y_predict)*100],'Recall':[recall_score(y_test, y_predict)*100]}))

In [ ]:
#SVM

In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')

clf.fit(X_train,y_train)
y_predict=clf.predict(X_test)

print('Accuracy score:',accuracy_score(y_test,y_predict))
print('confuion matrix:\n',confusion_matrix(y_test,y_predict))
print('Recall Score: ',recall_score(y_test, y_predict))
print('Precission Score: ',precision_score(y_test, y_predict))
print('F1 Score: ',f1_score(y_test, y_predict))
draw_cm(y_test, y_predict)
modelComp=modelComp.append(pd.DataFrame({'Model':['SVC'],'Accuracy':[accuracy_score(y_test,y_predict)*100],'Precission':[precision_score(y_test, y_predict)*100],'Recall':[recall_score(y_test, y_predict)*100]}))

In [ ]:
#Comapring Accuracy precission and Recall 
modelComp

- Considering the standard classification algorithm, SVM and Logistic Regression provided us with best Accuracy Precission and Recall values
- Updating the threshold to 0.4 for logistic is providing us with best recall values but its taking a hit on Precissiona dn overall accuracy as well.
- Considering that SVM has a better Overall values with Recall
- A 100% Recall score is good but we have a very small sample size and while using it on real world data might give us a unexpected or biased results. We should be looking at a model with best on all qalities and not a 100%.

In [ ]:
#META classifier

In [ ]:
#Stacking

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

#Base Learners
#knn_clf = KNeighborsClassifier(n_neighbors=3)
#svc_clf = SVC(kernel='linear')
#lr_clf = LogisticRegression(max_iter=10000)
#nb_Gau_clf=GaussianNB()
estimators = [('knn_clf',KNeighborsClassifier(n_neighbors=3)),
              ('nb_clf',GaussianNB()),
             ('lr_clf',LogisticRegression()),
             ('svc_clf',SVC(kernel='linear'))]
lr = LogisticRegression(max_iter=10000) # meta classifier
sclf = StackingClassifier(estimators=estimators, final_estimator=lr)

sclf.fit(X_train,y_train)
y_predict=sclf.predict(X_test)



print('Accuracy score:',accuracy_score(y_test,y_predict))
print('confuion matrix:\n',confusion_matrix(y_test,y_predict))
print('Recall Score: ',recall_score(y_test, y_predict))
print('Precission Score: ',precision_score(y_test, y_predict))
print('F1 Score: ',f1_score(y_test, y_predict))
draw_cm(y_test, y_predict)
modelComp=modelComp.append(pd.DataFrame({'Model':['Stacking Classifier(KNN,NB,LR,SVC)LR'],'Accuracy':[accuracy_score(y_test,y_predict)*100],'Precission':[precision_score(y_test, y_predict)*100],'Recall':[recall_score(y_test, y_predict)*100]}))

In [ ]:
y_pred_proba = sclf.predict_proba(X_test)[:, 1]
[fpr, tpr, thr] = roc_curve(y_test, y_pred_proba)

plt.figure()
plt.plot(fpr, tpr, color='coral', label='ROC curve Stacking - LogReg(area = %0.3f)' % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - specificity)', fontsize=14)
plt.ylabel('True Positive Rate (recall)', fontsize=14)
plt.title('Receiver operating characteristic (ROC) curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
modelComp

- With Stacking Classifier
- We can notice the stacking classifer has provided the best accuracy while trying to keep better Recall and preceission values
- SVC has 100% Recall and NB has 100% Precission but overall accuracy was slight low
- With stacking classifer we can observe it attained the highest Accuracy with next to best Precission and Recall. Only 1 value is missclassified from Status 1

In [ ]:
#Changing the final estimator to SVC in stacking

In [ ]:
estimators = [('knn_clf',KNeighborsClassifier(n_neighbors=3)),
              ('nb_clf',GaussianNB()),
             ('lr_clf',LogisticRegression()),
             ('svc_clf',SVC(kernel='linear'))]
lr = LogisticRegression() # meta classifier
sclf = StackingClassifier(estimators=estimators, final_estimator=SVC(kernel='linear'))

sclf.fit(X_train,y_train)
y_predict=sclf.predict(X_test)



print('Accuracy score:',accuracy_score(y_test,y_predict))
print('confuion matrix:\n',confusion_matrix(y_test,y_predict))
print('Recall Score: ',recall_score(y_test, y_predict))
print('Precission Score: ',precision_score(y_test, y_predict))
print('F1 Score: ',f1_score(y_test, y_predict))
draw_cm(y_test, y_predict)
modelComp=modelComp.append(pd.DataFrame({'Model':['Stacking Classifier(KNN,NB,LR,SVC)SVC'],'Accuracy':[accuracy_score(y_test,y_predict)*100],'Precission':[precision_score(y_test, y_predict)*100],'Recall':[recall_score(y_test, y_predict)*100]}))

In [ ]:
modelComp

In [ ]:
#Changing the final estimator to GaussianNB

In [ ]:
estimators = [('knn_clf',KNeighborsClassifier(n_neighbors=3)),
              ('nb_clf',GaussianNB()),
             ('lr_clf',LogisticRegression()),
             ('svc_clf',SVC(kernel='linear'))]
lr = LogisticRegression() # meta classifier
sclf = StackingClassifier(estimators=estimators, final_estimator=GaussianNB())

sclf.fit(X_train,y_train)
y_predict=sclf.predict(X_test)



print('Accuracy score:',accuracy_score(y_test,y_predict))
print('confuion matrix:\n',confusion_matrix(y_test,y_predict))
print('Recall Score: ',recall_score(y_test, y_predict))
print('Precission Score: ',precision_score(y_test, y_predict))
print('F1 Score: ',f1_score(y_test, y_predict))
draw_cm(y_test, y_predict)
modelComp=modelComp.append(pd.DataFrame({'Model':['Stacking Classifier(KNN,NB,LR,SVC)NB'],'Accuracy':[accuracy_score(y_test,y_predict)*100],'Precission':[precision_score(y_test, y_predict)*100],'Recall':[recall_score(y_test, y_predict)*100]}))

In [ ]:
modelComp

- On stacking classifiers, with different Final estimator we can observe the algorithm is trying its best to provide a best result.
- Stacking Classifier with Logistic is by far providing us with best results

In [ ]:
#Decission Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(criterion='entropy',max_depth=5,random_state=10,min_samples_leaf=5)
dt_model.fit(X_train, y_train)
y_predict=dt_model.predict(X_test)



print('Accuracy score:',accuracy_score(y_test,y_predict))
print('confuion matrix:\n',confusion_matrix(y_test,y_predict))
print('Recall Score: ',recall_score(y_test, y_predict))
print('Precission Score: ',precision_score(y_test, y_predict))
print('F1 Score: ',f1_score(y_test, y_predict))
draw_cm(y_test, y_predict)
modelComp=modelComp.append(pd.DataFrame({'Model':['Decision Tree'],'Accuracy':[accuracy_score(y_test,y_predict)*100],'Precission':[precision_score(y_test, y_predict)*100],'Recall':[recall_score(y_test, y_predict)*100]}))

In [ ]:
y_pred_proba = dt_model.predict_proba(X_test)[:, 1]
[fpr0, tpr0, thr0] = roc_curve(y_test, y_pred_proba)

plt.figure()
plt.plot(fpr, tpr, label='ROC curve Stacking - LogReg(area = %0.3f)' % auc(fpr, tpr))
plt.plot(fpr0, tpr0, label='ROC curve DecissionTree(area = %0.3f)' % auc(fpr0, tpr0))

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - specificity)', fontsize=14)
plt.ylabel('True Positive Rate (recall)', fontsize=14)
plt.title('Receiver operating characteristic (ROC) curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
modelComp

- Decission Tree with very minimal changes itself is able to provide us with a very better solution

In [ ]:
#Bagging Classifer

In [ ]:

from sklearn.ensemble import BaggingClassifier
bgclf = BaggingClassifier(base_estimator=dt_model, n_estimators=50, max_samples=.7)
bgclf = bgclf.fit(X_train, y_train)

y_predict=bgclf.predict(X_test)



print('Accuracy score:',accuracy_score(y_test,y_predict))
print('confuion matrix:\n',confusion_matrix(y_test,y_predict))
print('Recall Score: ',recall_score(y_test, y_predict))
print('Precission Score: ',precision_score(y_test, y_predict))
print('F1 Score: ',f1_score(y_test, y_predict))
draw_cm(y_test, y_predict)
modelComp=modelComp.append(pd.DataFrame({'Model':['BaggingClassifier'],'Accuracy':[accuracy_score(y_test,y_predict)*100],'Precission':[precision_score(y_test, y_predict)*100],'Recall':[recall_score(y_test, y_predict)*100]}))

In [ ]:
y_pred_proba = bgclf.predict_proba(X_test)[:, 1]
[fpr1, tpr1, thr1] = roc_curve(y_test, y_pred_proba)

plt.figure()
plt.plot(fpr, tpr, label='ROC curve Stacking - LogReg(area = %0.3f)' % auc(fpr, tpr))
plt.plot(fpr0, tpr0, label='ROC curve DecissionTree(area = %0.3f)' % auc(fpr0, tpr0))
plt.plot(fpr1, tpr1, label='ROC curve Bagging - DT(area = %0.3f)' % auc(fpr1, tpr1))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - specificity)', fontsize=14)
plt.ylabel('True Positive Rate (recall)', fontsize=14)
plt.title('Receiver operating characteristic (ROC) curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
modelComp

- With Bagging Classifer we can observe it is able to provide us best Accuracy, without hampering too much on Precission and Recall rate

In [ ]:
#Applying Ensemble Model

In [ ]:
#Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfclf = RandomForestClassifier(n_estimators = 50)
rfclf.fit(X_train, y_train)

y_predict=rfclf.predict(X_test)



print('Accuracy score:',accuracy_score(y_test,y_predict))
print('confuion matrix:\n',confusion_matrix(y_test,y_predict))
print('Recall Score: ',recall_score(y_test, y_predict))
print('Precission Score: ',precision_score(y_test, y_predict))
print('F1 Score: ',f1_score(y_test, y_predict))
draw_cm(y_test, y_predict)
modelComp=modelComp.append(pd.DataFrame({'Model':['Random Forest'],'Accuracy':[accuracy_score(y_test,y_predict)*100],'Precission':[precision_score(y_test, y_predict)*100],'Recall':[recall_score(y_test, y_predict)*100]}))

In [ ]:
y_pred_proba = rfclf.predict_proba(X_test)[:, 1]
[fpr2, tpr2, thr2] = roc_curve(y_test, y_pred_proba)

plt.figure()
plt.plot(fpr, tpr, label='ROC curve Stacking - LogReg(area = %0.3f)' % auc(fpr, tpr))
plt.plot(fpr0, tpr0, label='ROC curve DecissionTree(area = %0.3f)' % auc(fpr0, tpr0))
plt.plot(fpr1, tpr1, label='ROC curve Bagging - DT(area = %0.3f)' % auc(fpr1, tpr1))
plt.plot(fpr2, tpr2, label='ROC curve RandomForest(area = %0.3f)' % auc(fpr2, tpr2))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - specificity)', fontsize=14)
plt.ylabel('True Positive Rate (recall)', fontsize=14)
plt.title('Receiver operating characteristic (ROC) curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
modelComp

- Random Forest & Bagging Classifier is able to provide us with best solution so far with 93.2% accuracy, 93% precission and 97.67% Recall.
- Total 4 values are missclassifed but only 1 missclassification for status 1

In [ ]:
# ADABOOST

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
abcl = AdaBoostClassifier( n_estimators= 20) 
abcl.fit(X_train, y_train)
y_predict=abcl.predict(X_test)



print('Accuracy score:',accuracy_score(y_test,y_predict))
print('confuion matrix:\n',confusion_matrix(y_test,y_predict))
print('Recall Score: ',recall_score(y_test, y_predict))
print('Precission Score: ',precision_score(y_test, y_predict))
print('F1 Score: ',f1_score(y_test, y_predict))
draw_cm(y_test, y_predict)
modelComp=modelComp.append(pd.DataFrame({'Model':['Ada Boost'],'Accuracy':[accuracy_score(y_test,y_predict)*100],'Precission':[precision_score(y_test, y_predict)*100],'Recall':[recall_score(y_test, y_predict)*100]}))

In [ ]:
y_pred_proba = abcl.predict_proba(X_test)[:, 1]
[fpr3_0, tpr3_0, thr3_0] = roc_curve(y_test, y_pred_proba)

plt.figure(figsize=(8,8))
plt.plot(fpr, tpr, label='ROC curve Stacking - LogReg(area = %0.3f)' % auc(fpr, tpr))
plt.plot(fpr0, tpr0, label='ROC curve DecissionTree(area = %0.3f)' % auc(fpr0, tpr0))
plt.plot(fpr1, tpr1, label='ROC curve Bagging - DT(area = %0.3f)' % auc(fpr1, tpr1))
plt.plot(fpr2, tpr2, label='ROC curve RandomForest(area = %0.3f)' % auc(fpr2, tpr2))
plt.plot(fpr3_0, tpr3_0, label='ROC curve AdaBost(area = %0.3f)' % auc(fpr3_0, tpr3_0))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - specificity)', fontsize=14)
plt.ylabel('True Positive Rate (recall)', fontsize=14)
plt.title('Receiver operating characteristic (ROC) curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
modelComp

In [ ]:
#Gradient Boost

In [ ]:

from sklearn.ensemble import GradientBoostingClassifier
gbcl = GradientBoostingClassifier(n_estimators = 1000, learning_rate = 0.001)
gbcl = gbcl.fit(X_train, y_train)
y_predict=gbcl.predict(X_test)



print('Accuracy score:',accuracy_score(y_test,y_predict))
print('confuion matrix:\n',confusion_matrix(y_test,y_predict))
print('Recall Score: ',recall_score(y_test, y_predict))
print('Precission Score: ',precision_score(y_test, y_predict))
print('F1 Score: ',f1_score(y_test, y_predict))
draw_cm(y_test, y_predict)
modelComp=modelComp.append(pd.DataFrame({'Model':['GradientBoostingClassifier'],'Accuracy':[accuracy_score(y_test,y_predict)*100],'Precission':[precision_score(y_test, y_predict)*100],'Recall':[recall_score(y_test, y_predict)*100]}))

In [ ]:
y_pred_proba = gbcl.predict_proba(X_test)[:, 1]
[fpr3, tpr3, thr3] = roc_curve(y_test, y_pred_proba)

plt.figure(figsize=(8,4))
plt.plot(fpr, tpr, label='ROC curve Stacking - LogReg(area = %0.3f)' % auc(fpr, tpr))
plt.plot(fpr0, tpr0, label='ROC curve DecissionTree(area = %0.3f)' % auc(fpr0, tpr0))
plt.plot(fpr1, tpr1, label='ROC curve Bagging - DT(area = %0.3f)' % auc(fpr1, tpr1))
plt.plot(fpr2, tpr2, label='ROC curve RandomForest(area = %0.3f)' % auc(fpr2, tpr2))
plt.plot(fpr3_0, tpr3_0, label='ROC curve AdaBost(area = %0.3f)' % auc(fpr3_0, tpr3_0))
plt.plot(fpr3, tpr3, label='ROC curve GradientBoost(area = %0.3f)' % auc(fpr3, tpr3))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - specificity)', fontsize=14)
plt.ylabel('True Positive Rate (recall)', fontsize=14)
plt.title('Receiver operating characteristic (ROC) curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
modelComp

- Cosiderin all the Model's we can clearly notice the Ensemble models are providing better overall results
- Among the Classifers, Random Forest, AdaBoost, Bagging classifiers has provided us with best Accuracy, precission and recall in a single package. Mostly are based on decission trees.
- ROC curve for AdaBoost, Random Forest and Stacking - with final estimator as logistic regression has best AUC. If we observe the shape of the ROC for these graphs we can notice a slight different patttern for those. Classifiers that give curves closer to the top-left corner indicate a better performance.
- ROC curve for Decission tree is a lot diffeent then other models, the reason why it occurs in a decision tree is that you often do binary splits; this is efficient computationally, but only gives 2^n groupings. Unless your n number of splits are very large, you'll only have 16/32/64/128 groups, whereas if you used an algorithm such as logistic regression and used continous variables, your prediction would fall in the continous range between 0 and 1. As our model can only provide discrete predictions, rather than a continous score. This can often be remedied by adding more samples to your dataset, having more continous features in the model, more features in general or using a model specification that provides a continous prediction output.
- Considering few of the top models with best accuracy and recall, the ROC curve (area), we can can notice AdaBoost, Random Forest and Stacking - LogReg have close values, but Random Forest has much better overall accuracy along with AUC and hence we are considering it as our best model for the Task.